In [115]:
from custom.pipeline import ClassicPipeline, TransformerPipeline, txt2list, PreProcess
from custom.embeddings_generation import EmbeddingGenerator
from custom.sentiment import BiLSTMClassifier
import spacy
import torch
import pandas as pd
import math

In [116]:
IMDB = pd.read_csv('/Users/sergicastellasape/Repos/zeta-alpha/datasets/IMDB/IMDB_dataset.csv')

In [117]:
def sentiment2tensor(sent_list):
    logits = []
    for sent in sent_list:
        if sent == 'positive':
            logits.append([1, 0])
        elif sent == 'negative':
            logits.append([0, 1])
        else:
            raise ValueError("A sentiment wasn't positive or negative!")
    return torch.Tensor(logits)
    
def max_pooling(input_tensors, dim=0):
    # input tensors have shape (n, EmbDimension), 
    # this is problematic for now because this operation cannot be parallelized on GPU
    tensor, _ = torch.max(input_tensors, dim=0)
    return tensor

In [118]:
preprocess = PreProcess()
generator = EmbeddingGenerator(pool_function=max_pooling)

In [123]:
batch_size = 4
train_test_ratio = 0.8
max_epochs = 3
sentset_size = 2
num_layers = 2
embedding_size = 768
hidden_size = 768

n_batches = math.floor(len(IMDB) / batch_size)
batch_indices = torch.randperm(n_batches*batch_size).reshape(-1, batch_size)

split = math.floor(train_test_ratio*n_batches)
train_indices = batch_indices[0:split]
test_indices  = batch_indices[split:-1]


model = BiLSTMClassifier(embedding_size, hidden_size, sentset_size, num_layers, batch_size, dropout=0.)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

for epoch in range(max_epochs):
    for indices in batch_indices:
        i = [int(idx) for idx in indices]
        batch_sequence = IMDB['review'][i].tolist()
        batch_target_sent = IMDB['sentiment'][i].tolist()
        batch_target = sentiment2tensor(batch_target_sent)
        
        batch_tensors, indices_chunk = preprocess.forward(batch_sequence)
        #compressed, mask = generator.forward(batch_tensors, indices_chunk)
        
        batch_prediction = model.forward(batch_tensors)
        L = model.loss(batch_prediction, batch_target)
        L.backward()
        optimizer.step()
        raise ValueError('stop')

ValueError: stop

In [124]:
L

tensor(0.7246, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)

TypeError: floor(): argument 'input' (position 1) must be Tensor, not float